In [1]:
# %%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install "tensorflow-text==2.8.*"
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm

In [5]:
import pandas as pd
import os

In [6]:
from tqdm.notebook import tqdm
import codecs
import pandas as pd
tqdm.pandas()
import string
from multiprocessing import  Pool
import numpy as np

import random
import bokeh
import bokeh.models
import bokeh.plotting
import numpy as np
import os
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise

from simpleneighbors import SimpleNeighbors
from tqdm import tqdm
from tqdm import trange

2022-04-25 12:17:36.394925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 12:17:36.395003: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:

def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [8]:
os.listdir()

['__pycache__',
 'NLP Full.ipynb',
 'dummy',
 'Post_processing and Classiffication.ipynb',
 'embedder.py',
 '.ipynb_checkpoints',
 'splitData']

In [9]:
path="./splitData"

In [10]:
os.listdir(path)

['not_censored.csv', 'censored.csv', '.~lock.censored.csv#']

In [11]:
df_censored=pd.read_csv(path+"/censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
]).dropna()

In [12]:
df_censored = df_censored.drop(df_censored.index[df_censored['text'] == "text"])

In [13]:
len(df_censored)

38965

In [14]:
cols=["mid","text","created_at","deleted_last_seen","permission_denied"]

In [15]:
df_uncensored=pd.read_csv(path+"/not_censored.csv",names=["mid","text","created_at","deleted_last_seen","permission_denied"
],nrows=38965*2).dropna()

In [16]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
    return model(input)[-1]

def embed_text_list(input):
    return model(input).numpy()

2022-04-25 12:17:50.991976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 12:17:50.992550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 12:17:50.992694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-25 12:17:50.992814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-25 12:17:50.992932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [17]:
def parallel_guse(df):
    #df["guse"] = df["text"].progress_apply(lambda x:embed_text(str(x)))
    df["guse"] = embed_text_list(df["text"].to_list()).tolist()
    return df

In [18]:
# df_test=parallelize_dataframe(df=df_censored[:200], func=parallel_guse, n_cores=6)

In [19]:
# parallel_guse(df_censored[:200])

In [20]:
def serial_dataframe(df, func, n_splits=2):
    df_split = np.array_split(df, n_splits)
    df = pd.DataFrame()
    for df_s in tqdm(df_split):
        df_rs=func(df_s)
        df = pd.concat([df,df_s])
    return df

In [21]:
df_censored=serial_dataframe(df=df_censored, func=parallel_guse, n_splits=20)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:23<00:00,  4.18s/it]


In [22]:
df_uncensored=serial_dataframe(df=df_uncensored, func=parallel_guse, n_splits=20*2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:28<00:00,  3.72s/it]


In [31]:
df_uncensored.head()

,mid,text,created_at,deleted_last_seen,permission_denied,guse
0,mCClUNCqwe,转发微博,2012-01-03 02:02:27,False,False,"[0.10424910485744476, 0.044644154608249664, 0...."
1,mRsOcOLTlc,uK3RXUYW3： u0AGMTTVD： ！！！！！！！！,2012-01-03 01:17:39,False,False,"[-0.004717719741165638, -0.06476743519306183, ..."
2,mH44qG6iUm,求一切順利,2012-01-03 01:15:36,False,False,"[0.0962323397397995, -0.0058897421695292, 0.02..."
3,mZmwFtOdVX,想要uK3RXUYW3： ukn： 全都想要啊QAQ,2012-01-03 01:12:55,False,False,"[0.00019055344455409795, -0.026424074545502663..."
4,mQkLJSl8bf,ukn： uMLLV3ZCO： 转发微博,2012-01-03 01:10:42,False,False,"[0.015181104652583599, 0.01389026828110218, 0...."


In [39]:
df_data=pd.concat([df_censored.loc[:,['permission_denied','guse']],df_uncensored.loc[:,['permission_denied','guse']]])

In [46]:
df_data["permission_denied"]=df_data["permission_denied"].astype(bool).astype(int)

In [64]:
X=df_data["guse"].to_numpy()
X=np.vstack(X)

In [50]:
Y=df_data["permission_denied"].values

In [65]:
Y.shape

(116725,)

In [52]:
Y

array([1, 1, 1, ..., 0, 0, 0])

In [93]:
# from pprint import pprint

# import sklearn.datasets
# import sklearn.metrics

# import autosklearn.classification

FileNotFoundError: [Errno 2] No such file or directory: '/home/edwinjose/Documents/nlp/lib/python3.8/site-packages/scikit_learn-1.0.2.dist-info/METADATA'

In [77]:
# !pip uninstall hpsklearn -y

Found existing installation: hpsklearn 0.1.0
Uninstalling hpsklearn-0.1.0:
  Successfully uninstalled hpsklearn-0.1.0


In [88]:
# !pip uninstall hyperopt==0.2.5 -y

In [89]:
# from hpsklearn import HyperoptEstimator, any_classifier
# # from sklearn.datasets import fetch_mldata
# from hyperopt import tpe
# import numpy as np


In [91]:

# estim = HyperoptEstimator( classifier=any_classifier('clf'),  
#                             algo=tpe.suggest, trial_timeout=300)


# X=X
# y=Y

# test_size = int( 0.2 * len( y ) )
# # np.random.seed( seed )
# # np.random.RandomState(2)
# indices = np.random.permutation(len(X))
# X_train = X[ indices[:-test_size]]
# y_train = y[ indices[:-test_size]]
# X_test = X[ indices[-test_size:]]
# y_test = y[ indices[-test_size:]]

# estim = HyperoptEstimator( classifier=any_classifier('clf'),  
#                             algo=tpe.suggest, trial_timeout=300)

# estim.fit( X_train, y_train )

# print( estim.score( X_test, y_test ) )
# # <<show score here>>
# print( estim.best_model() )

  0%|                                                                                                      | 0/1 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'